In [ ]:
!pip install -r ../requirements.txt

In [1]:
import sys
sys.path.insert(0, "../")

# Downloading and processing data

There are two ways you can obtain OSM data in osm.pbf format:  
- Download it yourself: [parts of the world](https://download.geofabrik.de/), [cities](https://download.bbbike.org/osm/bbbike/), [adjustable area](https://extract.bbbike.org/) (via mail), [adjustable area](https://export.hotosm.org/en/v3/) (online), [planet](https://planet.maps.mail.ru/pbf/)
- Let the program download it for you

If the map is downloaded you can specify the filename:

In [ ]:
from visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
filename = "../maps/kozlovo.osm.pbf"
bbox = [36.2, 56.5, 36.7, 56.7]
vgraph.compute_geometry(bbox=bbox, filename=filename)

Or, alternatively, you can only specify the bounding box, and the map will be downloaded automatically ([curl](https://curl.se/) & [osmosis](https://wiki.openstreetmap.org/wiki/Osmosis) required):

In [ ]:
bbox = [34, 59, 34.2, 59.1]
vgraph.compute_geometry(bbox=bbox)

Data inside this area can be processed using VisibilityGraph with chosen or default parameters.  
If not specified, optimal parameters will be computed by the algorithm.

In [ ]:
vgraph.prune_geometry(epsilon_polygon=0.003,
                       epsilon_linestring=0.001,
                       bbox_comp=10)

Computed data can also be saved in .h5 file to skip data processing the next time:

In [ ]:
vgraph.save_geometry("../maps/user_area.h5")

# Using precomputed data and building visibility graph

In [2]:
from visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
vgraph.load_geometry("../maps/user_area.h5")

In [3]:
a = vgraph.polygons.geometry

In [7]:
a

0    (((34.1773601, 59.0005708), (34.158443, 59.001...
1    (((34.0228613, 59.0155847), (34.0388017, 59.00...
2    (((34.1773601, 59.0005708), (34.1917453, 59.00...
3    (((34.138577, 59.0398371), (34.1413021, 59.044...
4    (((34.1255882, 59.0993149), (34.077523, 59.097...
5    (((34.1049888, 59.0525599), (34.1128853, 59.04...
6    (((34.0228613, 59.0155847), (34.0388017, 59.00...
7    (((34.0342482, 59.0001777), (34.0554437, 59.00...
8    (((34.0004138, 59.0764101), (34.0141467, 59.07...
Name: geometry, dtype: object

In [5]:
a[0]

(((34.1773601, 59.0005708),
  (34.158443, 59.0012154),
  (34.1270542, 59.0129031),
  (34.1161537, 59.0108261),
  (34.1194153, 59.0171561),
  (34.1495166, 59.0150044),
  (34.1502033, 59.0183625),
  (34.1378437, 59.0201297),
  (34.1272688, 59.0308504),
  (34.1354441, 59.0373975),
  (34.1404438, 59.0315681),
  (34.1556964, 59.0302013),
  (34.1732059, 59.0356776),
  (34.1989551, 59.0252542),
  (34.1917453, 59.0033371),
  (34.1773601, 59.0005708)),)

In [6]:
a[2]

(((34.1773601, 59.0005708),
  (34.1917453, 59.0033371),
  (34.1989551, 59.0252542),
  (34.1732059, 59.0356776),
  (34.1556964, 59.0302013),
  (34.1404438, 59.0315681),
  (34.1354441, 59.0373975),
  (34.1272688, 59.0308504),
  (34.1378437, 59.0201297),
  (34.1502033, 59.0183625),
  (34.1495166, 59.0150044),
  (34.1194153, 59.0171561),
  (34.1161537, 59.0108261),
  (34.1270542, 59.0129031),
  (34.158443, 59.0012154),
  (34.1773601, 59.0005708)),)

Visibility graph can be built and (optionally) saved as networkx graph and (optionally) visualised using mplleaflet:

In [ ]:
%%time
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = vgraph.build_graph(inside_percent=0,
                            multiprocessing=False,
                            graph=True,
                            map_plot=map_plot)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

VisibilityGraph may also be used to find incident edges for a single point.  
This feature is used for pathfinding without graph building:

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

start = ((34.02, 59.01), None, None, None, None)
incidents = vgraph.incident_vertices(start)

fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)